# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mont-dore,-22.2833,166.5833,84.99,99,60,13.91,NC,1708486149
1,1,grytviken,-54.2811,-36.5092,36.90,85,100,10.89,GS,1708486149
2,2,naze,28.3667,129.4833,74.32,75,32,16.31,JP,1708486149
3,3,port-aux-francais,-49.3500,70.2167,46.15,90,100,35.90,TF,1708486149
4,4,aykhal,66.0000,111.5000,-9.87,99,93,6.06,RU,1708486149


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df['Humidity'].astype(float)
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size=humidity,
    scale=1,
    color='City'
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,_size)

In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] < 35)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,albany,42.6001,-73.9662,26.58,65,4,1.99,US,1708486160
179,179,smithers,54.7804,-127.1743,26.92,86,0,3.44,CA,1708486175
383,383,thunder bay,48.4001,-89.3168,26.58,100,20,3.44,CA,1708486043
523,523,torrington,41.8007,-73.1212,26.11,70,6,1.48,US,1708486120


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.loc[:, ['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
75,albany,US,42.6001,-73.9662,65,
179,smithers,CA,54.7804,-127.1743,86,
383,thunder bay,CA,48.4001,-89.3168,100,
523,torrington,US,41.8007,-73.1212,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:

# Set parameters to search for a hotel
radius = 10000
params = {
    'type':'accommodation.hotel',
    'radius': radius,
    'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, 'Lat']
    lng = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
smithers - nearest hotel: Sunshine Inn Hotel
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
torrington - nearest hotel: Yankee Pedlar Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
75,albany,US,42.6001,-73.9662,65,No hotel found
179,smithers,CA,54.7804,-127.1743,86,Sunshine Inn Hotel
383,thunder bay,CA,48.4001,-89.3168,100,Hampton Inn & Suites by Hilton Thunder Bay
523,torrington,US,41.8007,-73.1212,70,Yankee Pedlar Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
humidity = hotel_df['Humidity'].astype(float)
vaca_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = humidity,
    scale = 1,
    color = 'City',
    hover_cols = ['Lng','Lat','City','Humidity','Hotel Name','Country']
)
# How to format hover tool tip > https://discourse.holoviz.org/t/add-an-extra-field-when-hovering-in-hvplot-scatter/2331
# Display the map
vaca_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,_size,Humidity,Hotel Name,Country)